In [1]:
import numpy as np
from utils.config import process_config
from utils import factory

from matplotlib import pyplot as plt

import pandas as pd
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
import tensorflow as tf

Load a configuration file and data

In [ ]:
config, _ = process_config("configs/wae.json")

data = factory.create("data_loaders."+config.data['name'])(config)

Plot training losses and indicators

In [ ]:
log = pd.read_csv(config.summary_dir+"log.csv",sep=';',index_col=0)
res = pd.read_csv(config.summary_dir+"results.csv",sep=';',index_col=0)

log.plot()
plt.figure()
res.plot()

Load the model <br\>
Uncomment first lines to load new model

In [4]:
#del model 
#tf.reset_default_graph()

model = factory.create("models."+config.model['name'])(config)

Evaluate the model with respect to the dataset <br\>
Change checkpoint if you want to load model at a specific iteration

In [ ]:
N = 100000

# checkpoint = "experiments/" + config.exp_name + "/checkpoint/-53016"
checkpoint = None

fake_labels = None
if 'label' in config.data:
    fake_labels = data.sample_random_labels(N)
noise = model.sample_pz(N) if config.model['name'] =="wae" else model.sample_noise(N) 
gen_input = [model.sample_pz(N), fake_labels] #, fake_masks

with tf.Session() as sess:
    model.load(sess, checkpoint)
    fake_curves = model.generate_curves(sess,gen_input)

print("evaluating ...") 

fake_curves = data.inverse_transform(fake_curves, fake_labels)

results = data.test_similarity(fake_curves)
mean_tests = results["tests"] 
fake_inds =  results["indicators"]

print(mean_tests)
print(mean_tests.sum(1))

Plot the global mean of real and generated curves

In [ ]:
plt.rcParams["figure.figsize"] = (15,8)
plt.plot(data.raw_data.mean())
plt.plot(np.mean(fake_curves['data'],0))

Plot random generated curves

In [ ]:
ind = np.random.choice(1000)
plt.plot(fake_curves['data'][ind])

Plot indicator distribution

In [ ]:
plt.rcParams["figure.figsize"] = (15,10)
i=1
for ind in fake_inds.columns:
    plt.subplot(3,2,i)
    if ind=="sens_clim" or ind=="sens_chauf" or ind=="ind_res2":
        plt.hist(data.indicators[ind], alpha=0.5, label="real",normed=True)
        plt.hist(fake_inds[ind], alpha=0.5, label="real",normed=True)
    else:
        sns.distplot(np.arcsinh(data.indicators[ind]), kde=True, label="real")
        sns.distplot(np.arcsinh(fake_inds[ind]), kde=True, label="fake")
    i+=1

Evaluate WAE model

In [ ]:
N = 10000
test_curves = data.test_data[:N]
train_curves = data.train_data[:N]

with tf.Session() as sess:
    model.load(sess, checkpoint)
    train_res = model.reconstruct_curves(sess,train_curves)
    test_res = model.reconstruct_curves(sess,test_curves)
    
print("train reconstruction : "+ str(train_res['rec_loss']))
print("test reconstruction : "+ str(test_res['rec_loss']))

Compare reconstructed train curves

In [ ]:
plt.rcParams["figure.figsize"] = (15,8)
plt.plot(train_curves.mean(0))
plt.plot(train_res['rec_curves'].mean(0))

Compare reconstructed test curves

In [ ]:
plt.plot(test_curves.mean(0))
plt.plot(test_res['rec_curves'].mean(0))